In [1]:
import torch
import torch.nn as nn
from torchinfo import summary
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')

In [3]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [4]:
df.shape

(569, 33)

In [5]:
df.drop(columns=['id', 'Unnamed: 32'], inplace=True)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [8]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [10]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [11]:
X_train_tensor.shape

torch.Size([455, 30])

In [12]:
y_train_tensor.shape

torch.Size([455])

In [28]:
loss_function = nn.BCELoss()

In [17]:
class ANN(nn.Module):

    def __init__(self, num_features):

        super().__init__()
        self.linear = nn.Linear(num_features,1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, features):
        out = self.linear(features)
        out = self.sigmoid(out)
    def loss_function(self, y_pred, y):
        # Clamp predictions to avoid log(0)
        epsilon = 1e-7
        y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)
    
        # Calculate loss
        loss = -(y_train_tensor * torch.log(y_pred) + (1 - y_train_tensor) * torch.log(1 - y_pred)).mean()
        return loss
        return out

In [18]:
learning_rate = 0.1
epochs = 25

In [32]:
model = ANN(X_train_tensor.shape[1])

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)


# define loop
for epoch in range(epochs):

  # forward pass
  y_pred = model(X_train_tensor)

  # loss calculate
  loss = loss_function(y_pred, y_train_tensor.reshape(-1, 1))

  # backward pass
  loss.backward()

  # parameters update
  with torch.no_grad():
    model.linear.weight -= learning_rate * model.linear.weight.grad
    model.linear.bias -= learning_rate * model.linear.bias.grad

  # zero gradients
  model.linear.weight.grad.zero_()
  model.linear.bias.grad.zero_()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.7216932773590088
Epoch: 2, Loss: 0.5419870018959045
Epoch: 3, Loss: 0.44309425354003906
Epoch: 4, Loss: 0.3820575177669525
Epoch: 5, Loss: 0.34067949652671814
Epoch: 6, Loss: 0.31067267060279846
Epoch: 7, Loss: 0.28780657052993774
Epoch: 8, Loss: 0.2697145342826843
Epoch: 9, Loss: 0.2549744248390198
Epoch: 10, Loss: 0.24268195033073425
Epoch: 11, Loss: 0.2322351038455963
Epoch: 12, Loss: 0.2232174277305603
Epoch: 13, Loss: 0.21533173322677612
Epoch: 14, Loss: 0.20835988223552704
Epoch: 15, Loss: 0.2021380513906479
Epoch: 16, Loss: 0.19654060900211334
Epoch: 17, Loss: 0.19146957993507385
Epoch: 18, Loss: 0.18684719502925873
Epoch: 19, Loss: 0.18261098861694336
Epoch: 20, Loss: 0.1787099987268448
Epoch: 21, Loss: 0.1751023530960083
Epoch: 22, Loss: 0.17175309360027313
Epoch: 23, Loss: 0.16863290965557098
Epoch: 24, Loss: 0.16571703553199768
Epoch: 25, Loss: 0.1629841923713684


In [34]:
X_test_tensor = torch.tensor(X_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_train, dtype=torch.float32)

In [37]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.9).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.5657722353935242
